In [15]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data87787


In [16]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [17]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/
  Using cached https://mirror.baidu.com/pypi/packages/d1/41/e6495bd7d3781cee623ce23ea6ac73282a373088fcd0ddc809a047b18eae/beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
  Using cached https://mirror.baidu.com/pypi/packages/36/69/d82d04022f02733bf9a72bc3b96332d360c0c5307096d76f6bb7489f7e57/soupsieve-2.2.1-py3-none-any.whl (33 kB)
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.


In [18]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [10]:
import pandas as pd
import numpy as np
data=pd.read_csv('data/data87787/train.csv')
result=pd.read_csv('data/data87787/submission.csv')
b=list(data)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [11]:
data['order_pay_time']=pd.to_datetime(data['order_pay_time'])
##构建用户购买时间特征，包括上一次购买时间
data1=data[data['order_pay_time']<='2013-07-31']
data1 = data1.drop_duplicates(['order_id'])
data1  = data1.set_index(['customer_id','order_id'])
data1 = data1.sort_values(['customer_id','order_pay_time'])
data1['last_pay_time'] = data1['order_pay_time'].shift()
data1.reset_index(inplace=True)
data1['customer_id_before'] = data1['customer_id'].shift().fillna(0).astype(np.uint32)
def fill_row(x, y, a, b):
    if x != y:
        t = np.nan
    else:
        t = (a - b).days
    return t
data1['days_since_prior_order'] = data1.apply(lambda row: fill_row(row['customer_id'],row['customer_id_before'],row['order_pay_time'],row['last_pay_time']), axis=1)
del data1['customer_id_before']
del data1['last_pay_time']
data1['order_hour_of_day'] = pd.DatetimeIndex(data1['order_pay_time']).hour
data1['order_dow'] = pd.DatetimeIndex(data1['order_pay_time']).dayofweek
data1['count']=1

In [12]:
##将购买时间分为工作时间（9-18点）记为1和非工作时间段记为0
def datime(a):
	if a>9 and a<=18:
		t=1
	else:
		t=0
	return t
data2=data1[['customer_id','order_hour_of_day','count']]
data2['work_time'] = data2.apply(lambda row: datime(row['order_hour_of_day']), axis=1)
data2=data2.groupby(['customer_id','work_time']).agg({'count':'sum'})
data2.reset_index(inplace=True)
data2['lastid']=data2['customer_id'].shift().fillna(0).astype(np.uint32)
data2['lastworktime']=data2['work_time'].shift().fillna(0).astype(np.uint32)
data2['lastcount']=data2['count'].shift().fillna(0).astype(np.uint32)
def qs(a,b,c,d,e,f):
	if a==b:
		if e>f:
			t=c
		else:
			t=d
	else:
		t=c
	return t
data2['work_time2'] = data2.apply(lambda row: qs(row['customer_id'],row['lastid'],row['work_time'],row['lastworktime'],row['count'],row['lastcount']), axis=1)
data2=data2.groupby(['customer_id'])['work_time2'].mean()

In [13]:
train=pd.DataFrame()
train['customer_id']=data1['customer_id'].unique()
train=train[['customer_id']].sort_values('customer_id')
train['user_orderid_count'] = data1.groupby('customer_id')['order_id'].count().values 
train['user_days_since_prior_order_mean'] = data1.groupby('customer_id')['days_since_prior_order'].mean().values
train['user_days_since_prior_order_max'] = data1.groupby('customer_id')['days_since_prior_order'].max().values
train['user_days_since_prior_order_min'] = data1.groupby('customer_id')['days_since_prior_order'].min().values
train['user_use_hour']=list(data2)
del data2

In [14]:
data3=data1[['customer_id','order_dow','count']]
##将购买时间分为工作日（周一至周五）以及非工作日（周日周六）
def datime2(a):
	if a>=1 and a<=5:
		t=1
	else:
		t=0
	return t
data3['week_time'] = data3.apply(lambda row: datime2(row['order_dow']), axis=1)
data3=data3.groupby(['customer_id','week_time']).agg({'count':'sum'})
data3.reset_index(inplace=True)
data3['lastid']=data3['customer_id'].shift().fillna(0).astype(np.uint32)
data3['lastweektime']=data3['week_time'].shift().fillna(0).astype(np.uint32)
data3['lastcount']=data3['count'].shift().fillna(0).astype(np.uint32)
def qs2(a,b,c,d,e,f):
	if a==b:
		if e>f:
			t=c
		else:
			t=d
	else:
		t=c
	return t
data3['week_time2'] = data3.apply(lambda row: qs2(row['customer_id'],row['lastid'],row['week_time'],row['lastweektime'],row['count'],row['lastcount']), axis=1)
data3=data3.groupby(['customer_id'])['week_time2'].mean()
train['user_use_week']=list(data3)
del data3

In [ ]:
data2=data1.groupby(['customer_id'])['order_total_num'].sum()
train['order_total_num']=list(data2)
del data2
data2=data1.groupby('customer_id')['order_total_payment'].sum()
train['order_total_pay']=list(data2)
data2=data1.groupby('customer_id')['order_total_discount'].sum()
train['discount']=list(data2)
data2=data1.groupby(['customer_id'])['customer_gender'].max()
train['sex']=list(data2)
data2=data1.groupby('customer_id')['is_customer_rate'].mean()
train['is_customer_rate']=list(data2)
data2=data1[['customer_id','customer_province']]
data2['customer_province']=pd.factorize(data1['customer_province'])[0]
data2=data2.groupby('customer_id')['customer_province'].mean()
train['province']=list(data2)
train['user_days_since_prior_order_mean']=train['user_days_since_prior_order_mean'].fillna(1000)
train['user_days_since_prior_order_max']=train['user_days_since_prior_order_max'].fillna(1000)
train['user_days_since_prior_order_min']=train['user_days_since_prior_order_min'].fillna(1000)
train['sex']=train['sex'].fillna(3)
del data2
del data1

In [ ]:
train_y=pd.DataFrame()
train_y['customer_id']=train['customer_id'].unique()
train_y['y']=0
c=data[(data['order_pay_time']>='2013-08-1')&(data['order_pay_time']<'2013-08-31')]['customer_id'].unique()
for i in range(len(train_y)):
	if train_y['customer_id'][i] in c:
		train_y['y'][i]=1
train_y.set_index(['customer_id'], inplace=True)
train.set_index(['customer_id'], inplace=True)

In [ ]:
import xgboost as xgb
model=xgb.XGBClassifier()
#from sklearn.ensemble import RandomForestRegressor
#model=RandomForestRegressor()
model.fit(train,train_y)
del train
del train_y

In [ ]:
data1=data
data1 = data1.drop_duplicates(['order_id'])
data1  = data1.set_index(['customer_id','order_id'])
data1 = data1.sort_values(['customer_id','order_pay_time'])
data1['last_pay_time'] = data1['order_pay_time'].shift()
data1.reset_index(inplace=True)
data1['customer_id_before'] = data1['customer_id'].shift().fillna(0).astype(np.uint32)
def fill_row(x, y, a, b):
    if x != y:
        t = np.nan
    else:
        t = (a - b).days
    return t
data1['days_since_prior_order'] = data1.apply(lambda row: fill_row(row['customer_id'],row['customer_id_before'],row['order_pay_time'],row['last_pay_time']), axis=1)
del data1['customer_id_before']
del data1['last_pay_time']
data1['order_hour_of_day'] = pd.DatetimeIndex(data1['order_pay_time']).hour
data1['order_dow'] = pd.DatetimeIndex(data1['order_pay_time']).dayofweek
data1['count']=1
def datime(a):
	if a>9 and a<=18:
		t=1
	else:
		t=0
	return t
data2=data1[['customer_id','order_hour_of_day','count']]
data2['work_time'] = data2.apply(lambda row: datime(row['order_hour_of_day']), axis=1)
data2=data2.groupby(['customer_id','work_time']).agg({'count':'sum'})
data2.reset_index(inplace=True)
data2['lastid']=data2['customer_id'].shift().fillna(0).astype(np.uint32)
data2['lastworktime']=data2['work_time'].shift().fillna(0).astype(np.uint32)
data2['lastcount']=data2['count'].shift().fillna(0).astype(np.uint32)
def qs(a,b,c,d,e,f):
	if a==b:
		if e>f:
			t=c
		else:
			t=d
	else:
		t=c
	return t
data2['work_time2'] = data2.apply(lambda row: qs(row['customer_id'],row['lastid'],row['work_time'],row['lastworktime'],row['count'],row['lastcount']), axis=1)
data2=data2.groupby(['customer_id'])['work_time2'].mean()

train=pd.DataFrame()
train['customer_id']=data1['customer_id'].unique()
train=train[['customer_id']].sort_values('customer_id')
train['user_orderid_count'] = data1.groupby('customer_id')['order_id'].count().values 
train['user_days_since_prior_order_mean'] = data1.groupby('customer_id')['days_since_prior_order'].mean().values
train['user_days_since_prior_order_max'] = data1.groupby('customer_id')['days_since_prior_order'].max().values
train['user_days_since_prior_order_min'] = data1.groupby('customer_id')['days_since_prior_order'].min().values
train['user_use_hour']=list(data2)
del data2
data3=data1[['customer_id','order_dow','count']]
def datime2(a):
	if a>=1 and a<=5:
		t=1
	else:
		t=0
	return t
data3['week_time'] = data3.apply(lambda row: datime2(row['order_dow']), axis=1)
data3=data3.groupby(['customer_id','week_time']).agg({'count':'sum'})
data3.reset_index(inplace=True)
data3['lastid']=data3['customer_id'].shift().fillna(0).astype(np.uint32)
data3['lastweektime']=data3['week_time'].shift().fillna(0).astype(np.uint32)
data3['lastcount']=data3['count'].shift().fillna(0).astype(np.uint32)
def qs2(a,b,c,d,e,f):
	if a==b:
		if e>f:
			t=c
		else:
			t=d
	else:
		t=c
	return t
data3['week_time2'] = data3.apply(lambda row: qs2(row['customer_id'],row['lastid'],row['week_time'],row['lastweektime'],row['count'],row['lastcount']), axis=1)
data3=data3.groupby(['customer_id'])['week_time2'].mean()
train['user_use_week']=list(data3)
del data3
data2=data1.groupby(['customer_id'])['order_total_num'].sum()
train['order_total_num']=list(data2)
del data2
data2=data1.groupby('customer_id')['order_total_payment'].sum()
train['order_total_pay']=list(data2)
data2=data1.groupby('customer_id')['order_total_discount'].sum()
train['discount']=list(data2)
data2=data1.groupby(['customer_id'])['customer_gender'].max()
train['sex']=list(data2)
data2=data1.groupby('customer_id')['is_customer_rate'].mean()
train['is_customer_rate']=list(data2)
data2=data1[['customer_id','customer_province']]
data2['customer_province']=pd.factorize(data1['customer_province'])[0]
data2=data2.groupby('customer_id')['customer_province'].mean()
train['province']=list(data2)
del data2
del data1
train['user_days_since_prior_order_mean']=train['user_days_since_prior_order_mean'].fillna(1000)
train['user_days_since_prior_order_max']=train['user_days_since_prior_order_max'].fillna(1000)
train['user_days_since_prior_order_min']=train['user_days_since_prior_order_min'].fillna(1000)
train['sex']=train['sex'].fillna(3)
train.set_index(['customer_id'], inplace=True)

In [ ]:
jj=model.predict(train)
for i in range(len(result)):
	result['result'][i]=jj[i]
jj=pd.DataFrame(jj)
jj.index=result['customer_id']
jj.columns=['result']
jj.to_csv('result.csv')